In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
import numpy as np
from keras.src import Model

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

#initializing the CNN
classifier = Sequential()

#Step1: Convolution
classifier.add(Convolution2D(32, 3, 3, input_shape =(64, 64, 3), activation= "relu"))

#step2: Max Pooling
classifier.add(MaxPooling2D(pool_size=(2,2)))

#adding a second convolutional layer
classifier.add(Convolution2D(32, 3, 3, activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

#step3: Flattening
classifier.add(Flatten())

#step4: Full Connection
classifier.add(Dense(128, activation='relu'))


#output layer
classifier.add(Dense(1, activation='softmax'))

#compiling the CNN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


#Training datagen
train_datagen= ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True    )

#test datagen
test_datagen=ImageDataGenerator(rescale=1./255)


#Training Set & Test Set
train_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Programs/PBL-5 Data Set/train', target_size=(64, 64), batch_size=32, class_mode="binary")

test_set= test_datagen.flow_from_directory('/content/drive/MyDrive/Programs/PBL-5 Data Set/test', target_size=(64,64), batch_size=32, class_mode='binary')

In [ ]:
classifier.fit(train_set, steps_per_epoch=56, epochs=10, validation_data=train_set,validation_steps=500)

In [3]:
#Importing the image(Load Single Image From Dataset)
test_image =image.load_img('/content/drive/MyDrive/Programs/PBL-5 Data Set/test/highfive/10.jpg', target_size=(64,64))

In [4]:
#Setting Dimension
test_image=image.img_to_array(test_image)

In [5]:
#Set The Dimension
test_image=np.expand_dims(test_image, axis=0)

In [ ]:
#predict the correct image
result = classifier.predict(test_image)

In [ ]:
train_set.class_indices

In [ ]:
import cv2
import numpy as np
from tensorflow.python.keras.models import model_from_json
from google.colab.patches import cv2_imshow

emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# load json and create model
json_file = open('/content/drive/MyDrive/Programs/PBL5-2/AIA_GROUPNO13_Project/model/emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

emotion_model = model_from_json(loaded_model_json)

# load weights into new model
emotion_model.load_weights("/content/drive/MyDrive/Programs/PBL5-2/AIA_GROUPNO13_Project/model/emotion_model.h5")
print("Loaded model from disk")

# start the webcam feed
#cap = cv2.VideoCapture(0)


cap = cv2.VideoCapture("/content/drive/MyDrive/Programs/PBL5-2/AIA_GROUPNO13_Project/video3.mp4")

while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1280, 720))
    if not ret:
        break
    face_detector = cv2.CascadeClassifier('/content/drive/MyDrive/Programs/PBL5-2/AIA_GROUPNO13_Project/haarcascades/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # take each face available on the camera and Preprocess it
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)

        # predict the emotions
        emotion_prediction = emotion_model.predict_on_batch(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    #cv2.imshow('Emotion Detection', frame)
    cv2_imshow(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


https://www.linkedin.com/pulse/facial-emotion-recognition-opencv-deepface-step-by-step-m-